In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


###SELECT MODEL TO OPTIMIZE
model_num = 2

5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=20, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()'],
 ["['B_avg_SUB_ATT', 'B_Heavyweight_rank', 'ko_dif', 'B_current_lose_streak', 'B_Light Heavyweight_rank', 'B_Lightweight_rank', 'R_Flyweight_rank', 'R_Welterweight_rank', 'R_Middleweight_rank', 'avg_td_dif', 'B_avg_TD_pct', 'empty_arena', 'R_Weight_lbs', 'country', 'B_W

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4107
4107
4107


In [9]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
200,Andre Ewell,Anderson Dos Santos,-140,120,71.428571,120.000000,2019-06-22,"Greenville, South Carolina, USA",USA,Red,...,9.0,0.00,0.22,0.0,0.0,0.0,4.0,0.0,0.0,0
201,Andrea Lee,Montana De La Rosa,-185,160,54.054054,160.000000,2019-06-22,"Greenville, South Carolina, USA",USA,Red,...,12.0,0.33,0.41,0.0,1.0,2.0,0.0,0.0,0.0,0
202,Kevin Holland,Alessio Di Chirico,-230,190,43.478261,190.000000,2019-06-22,"Greenville, South Carolina, USA",USA,Red,...,4.0,0.00,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0
203,Dan Ige,Kevin Aguilar,120,-140,120.000000,71.428571,2019-06-22,"Greenville, South Carolina, USA",USA,Red,...,1.0,0.25,1.00,0.0,0.0,3.0,0.0,0.0,1.0,0
204,Ashley Yoder,Syuri Kondo,-140,120,71.428571,120.000000,2019-06-22,"Greenville, South Carolina, USA",USA,Red,...,0.0,0.20,0.00,0.0,1.0,8.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,2.0,0.00,0.50,0.0,0.0,0.0,0.0,0.0,0.0,1
4303,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,1.0,1.00,1.00,0.0,1.0,0.0,1.0,0.0,1.0,0
4304,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0
4305,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,0.60,0.00,0.0,0.0,6.0,0.0,0.0,0.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_td_attempted_bout,R_td_pct_bout,B_td_pct_bout,R_sub_attempts_bout,B_sub_attempts_bout,R_pass_bout,B_pass_bout,R_rev_bout,B_rev_bout,label
0,Robert Whittaker,Darren Till,-130,107,76.923077,107.000000,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Mauricio Rua,Rogerio Nogueira,-190,150,52.631579,150.000000,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Fabricio Werdum,Alexander Gustafsson,260,-335,260.000000,29.850746,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Carla Esparza,Marina Rodriguez,145,-182,145.000000,54.945055,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,Paul Craig,Gadzhimurad Antigulov,-137,110,72.992701,110.000000,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Dalcha Lungiambula,Dequan Townsend,-420,335,23.809524,335.000000,2019-06-29,"Minneapolis, Minnesota, USA",USA,Red,...,0.0,0.66,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0
196,Emily Whitmire,Amanda Ribas,-175,155,57.142857,155.000000,2019-06-29,"Minneapolis, Minnesota, USA",USA,Blue,...,5.0,0.00,0.4,0.0,2.0,0.0,3.0,1.0,0.0,1
197,Maurice Greene,Junior Albini,-125,105,80.000000,105.000000,2019-06-29,"Minneapolis, Minnesota, USA",USA,Red,...,0.0,0.00,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
198,Renato Moicano,Chan Sung Jung,-220,180,45.454545,180.000000,2019-06-22,"Greenville, South Carolina, USA",USA,Blue,...,0.0,0.00,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1


In [10]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)


D:\Anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [11]:
df_test
df_test.to_csv('test.csv')

In [12]:
#1. Set features
#2. Set Hyperparameters
#3. Set EV
#4. Remove Features

In [13]:
models[1][model_num]

"RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)"

In [14]:
test_model_name = models[0][model_num]
test_model = eval(models[1][model_num])
test_model_features = eval(models[2][model_num])
test_model_ev = eval(models[3][model_num])

In [15]:
old_test_model = test_model
old_test_model_features = test_model_features
old_test_model_ev = test_model_ev

In [16]:
print(df_test.columns.tolist()) #Keeps us from truncating

['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date', 'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',

In [17]:
#1. set features
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena', 'B_match_weightclass_rank', 'R_match_weightclass_rank', 
        "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank",
        "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 
        'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 
        'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", 
        "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 
        'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 
        'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 
        'B_Pound-for-Pound_rank', 'better_rank']

In [18]:
print(test_model_name)
print(test_model)
print(test_model_features)
print(test_model_ev)



model_2
RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,
                       random_state=75)
["R_Women's Featherweight_rank", 'B_odds', 'B_avg_SUB_ATT', 'R_total_rounds_fought', 'age_dif', 'R_win_by_Decision_Split', 'R_avg_TD_pct', 'empty_arena']
1


In [19]:
def save_model():
    score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)
    models[0][model_num] =  test_model_name 
    models[1][model_num] = test_model
    models[2][model_num] = test_model_features
    models[3][model_num] = test_model_ev
    models[4][model_num] = score    
    
    with open('models.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        for row in models:
            print("HI")
            writer.writerow(row)

    outfile.close()    

In [20]:
def print_model():
    print()
    print(test_model_name)
    print(test_model)
    print(test_model_features)
    print(test_model_ev)
    print()

In [21]:
keep_going = True
while(keep_going):
    #1. Set Features
    #get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False)
    test_model_features = (get_best_features(my_pos_features, test_model, df_train, test_model_features, label_train, odds_train, 
                                             min_ev=test_model_ev))
    print_model()
    save_model()
    #2 Set hyperparameters
    #def tune_hyperparameters(input_model, input_features, input_df, input_labels, odds_input):
    test_model = tune_hyperparameters(test_model, test_model_features, df_train, label_train, odds_train, 
                                      min_ev=test_model_ev)
    print_model()
    save_model()
    #3. Set EV
    #def tune_ev(input_model, input_features, input_df, input_labels, odds_input, verbose=False):
    test_model_ev = tune_ev(test_model, test_model_features, df_train, label_train, odds_train, verbose=False)

    old_test_model_features = test_model_features #This prevents
                                                  #an uneccesary loop
    print_model()
    save_model()
    #4. Remove Features
    #def remove_to_improve(cur_features, m, df, labels, odds, scale=False, min_ev = 0):
    test_model_features = remove_to_improve(test_model_features, test_model, df_train, label_train, odds_train, min_ev = test_model_ev)    
    keep_going = False
    if old_test_model != test_model:
        print("The hyperparameters are different")
        print("OLD:")
        print(old_test_model)
        print("NEW:")
        print(test_model)
        keep_going = True
        old_test_model = test_model
    if old_test_model_features != test_model_features:
        print("The features are different")
        print("OLD:")
        print(old_test_model_features)
        print("NEW:")
        print(test_model_features)
        keep_going = True
        old_test_model_features = test_model_features
    if old_test_model_ev != test_model_ev:
        print("The EV is different")
        print("OLD:")
        print(old_test_model_ev)
        print("NEW:")
        print(test_model_ev)
        keep_going = True
        old_test_model_ev = test_model_ev
    print_model()
    save_model()
    

Current best score is: 21557.47578260439
NO IMPROVEMENT
FINAL BEST SCORE: 21557.47578260439

model_2
RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,
                       random_state=75)
["R_Women's Featherweight_rank", 'B_odds', 'B_avg_SUB_ATT', 'R_total_rounds_fought', 'age_dif', 'R_win_by_Decision_Split', 'R_avg_TD_pct', 'empty_arena']
1



(3154, 8)

(3154,)

(3154, 2)

(200, 8)

...

(200, 8)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-130,107,0.484386,0.515614,0
1,-190,150,0.511276,0.488724,0
2,260,-335,0.494536,0.505464,0
3,145,-182,0.481560,0.518440,0
4,-137,110,0.516247,0.483753,0
...,...,...,...,...,...
195,-420,335,0.781755,0.218245,0
196,-175,155,0.550300,0.449700,1
197,-125,105,0.438512,0.561488,0
198,-220,180,0.700133,0.299867,1



          Number of matches: 200
          Number of bets: 159
          Number of winning bets: 78
          Number of losing bets: 81
          Number of underdog bets: 100
          Number of underdog wins: 38
          Number of underdog losses: 62
          Number of Favorite bets: 55
          Number of favorite wins: 38
          Number of favorite losses: 17
          Number of even bets: 4
          Number of even wins: 2
          Number of even losses: 2
          Profit: -52.76692453554472
          Profit per bet: -0.3318674499090863
          Profit per match: -0.26383462267772356
          
          
HI
HI
HI
HI
HI


In [22]:
#Get new score:
#def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels, test_odds, verbose=True):
score = evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(3154, 8)

(3154,)

(3154, 2)

(200, 8)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-130,107,0.484386,0.515614,0
1,-190,150,0.511276,0.488724,0
2,260,-335,0.494536,0.505464,0
3,145,-182,0.481560,0.518440,0
4,-137,110,0.516247,0.483753,0
...,...,...,...,...,...
195,-420,335,0.781755,0.218245,0
196,-175,155,0.550300,0.449700,1
197,-125,105,0.438512,0.561488,0
198,-220,180,0.700133,0.299867,1



          Number of matches: 200
          Number of bets: 159
          Number of winning bets: 78
          Number of losing bets: 81
          Number of underdog bets: 100
          Number of underdog wins: 38
          Number of underdog losses: 62
          Number of Favorite bets: 55
          Number of favorite wins: 38
          Number of favorite losses: 17
          Number of even bets: 4
          Number of even wins: 2
          Number of even losses: 2
          Profit: -52.76692453554472
          Profit per bet: -0.3318674499090863
          Profit per match: -0.26383462267772356
          
          


print(test_model_name) 
print(models[0][model_num])
print()
print(test_model)
print(eval(models[1][model_num]))
print()
print(test_model_features) 
print(eval(models[2][model_num]))
print()
print(test_model_ev)
print(eval(models[3][model_num]))

In [23]:
models[0][model_num] =  test_model_name 
models[1][model_num] = test_model
models[2][model_num] = test_model_features
models[3][model_num] = test_model_ev
models[4][model_num] = score

In [24]:
with open('models.csv', 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI


In [25]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [26]:
data

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20'],
 ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'DecisionTreeClassifier(max_depth=20, random_state=75)',
  "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)",
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',
  'GaussianNB()'],
 ["['B_avg_SUB_ATT', 'B_Heavyweight_rank', 'ko_dif', 'B_current_lose_streak', 'B_Light Heavyweight_rank', 'B_Lightweight_rank', 'R_Flyweight_rank', 'R_Welterweight_rank', 'R_Middleweight_rank', 'avg_td_dif', 'B_avg_TD_pct', 'empty_arena', 'R_Weight_lbs', 'country', 'B_W